In [22]:
import numpy as np
import pandas as pd
import mlflow

In [23]:
mlflow.set_tracking_uri('http://ec2-51-20-64-139.eu-north-1.compute.amazonaws.com:5000')
mlflow.set_experiment('exp 5 - ML Algos with HP Tuning')

2025/01/22 17:00:24 INFO mlflow.tracking.fluent: Experiment with name 'exp 5 - ML Algos with HP Tuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://neeraj-first-bucket/615931187569087796', creation_time=1737545423333, experiment_id='615931187569087796', last_update_time=1737545423333, lifecycle_stage='active', name='exp 5 - ML Algos with HP Tuning', tags={}>

In [24]:
!pip install xgboost


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
!pip install optuna


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import ADASYN
import matplotlib.pyplot as plt
import seaborn as sns
import optuna

In [27]:
df = pd.read_csv('dataset.csv')
df.dropna(inplace=True)
df.head()

,clean_comment,category
0,family mormon never tried explain still stare ...,1
1,buddhism much lot compatible christianity espe...,1
2,seriously say thing first get complex explain ...,-1
3,learned want teach different focus goal not wr...,0
4,benefit may want read living buddha living chr...,1


In [28]:
df['category'] = df['category'].map({-1:2, 0:0, 1:1})

In [29]:
ngram_range = (1,2)
max_features = 2000
X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2)

# vectorizer
vectorizer = TfidfVectorizer(max_features=max_features, ngram_range=ngram_range)
X_train_trf = vectorizer.fit_transform(X_train)
X_test_trf = vectorizer.transform(X_test)

# oversampling
adasyn = ADASYN(random_state=42)
X_train_trf, y_train = adasyn.fit_resample(X_train_trf, y_train)

In [30]:
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    if mlflow.active_run():
        mlflow.end_run()
        
    with mlflow.start_run(nested=True):
        mlflow.set_tag("mlflow.runName", f"{model_name}_Adasyn_Tfidf_Bigram")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        mlflow.log_param("algorithm", model_name)

        # model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # metrics
        accuracy = accuracy_score(y_test, y_pred)
        class_report = classification_report(y_test, y_pred, output_dict=True)

        mlflow.log_metric("accuracy", accuracy)
        for label, metrics in class_report.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        mlflow.sklearn.log_model(model, f"{model_name}_model")

In [31]:
# optuna objective function
def objective_xgboost(trial):
    n_estimators = trial.suggest_int("n_estimators", 50, 300)
    lr = trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int("max_depth", 3, 10)
    model = XGBClassifier(n_estimators=n_estimators, learning_rate=lr, max_depth=max_depth, random_state=42)

    return accuracy_score(y_test, model.fit(X_train_trf, y_train).predict(X_test_trf))

# run optuna
def run_optuna_xgboost():
    study = optuna.create_study(direction='maximize')
    study.optimize(objective_xgboost, n_trials=30)

    # run the best parameters and log the best model
    best_params = study.best_params
    best_model = XGBClassifier(n_estimators=best_params['n_estimators'], learning_rate=best_params['learning_rate'], max_depth=best_params['max_depth'], random_state=42)

    log_mlflow('XGBoost', best_model, X_train_trf, X_test_trf, y_train, y_test)
    

In [32]:
run_optuna_xgboost()

[I 2025-01-22 17:01:08,066] A new study created in memory with name: no-name-1286c6c1-79b4-4cf9-a800-92b62ea268c7


[I 2025-01-22 17:23:12,794] Trial 0 finished with value: 0.6003000136369835 and parameters: {'n_estimators': 290, 'learning_rate': 0.0007667165027578911, 'max_depth': 9}. Best is trial 0 with value: 0.6003000136369835.
[I 2025-01-22 17:46:01,487] Trial 1 finished with value: 0.6424382926496659 and parameters: {'n_estimators': 270, 'learning_rate': 0.00686957553489797, 'max_depth': 7}. Best is trial 1 with value: 0.6424382926496659.
[I 2025-01-22 17:49:36,133] Trial 2 finished with value: 0.7586253920632756 and parameters: {'n_estimators': 99, 'learning_rate': 0.08798507377590256, 'max_depth': 9}. Best is trial 2 with value: 0.7586253920632756.
[I 2025-01-22 18:05:58,060] Trial 3 finished with value: 0.5861175507977635 and parameters: {'n_estimators': 238, 'learning_rate': 0.0002459747126266874, 'max_depth': 8}. Best is trial 2 with value: 0.7586253920632756.
[I 2025-01-22 18:17:13,806] Trial 4 finished with value: 0.8047183962907405 and parameters: {'n_estimators': 276, 'learning_rate'

RestException: INVALID_PARAMETER_VALUE: The run 2db9457a89ba476db50b8e8040e82dec must be in 'active' lifecycle_stage.